# This is Keerthana's version of this notebook and adapted directly: 
https://github.com/AllonKleinLab/SPRING_dev/blob/aa52c405b6f15efd53c66f6856799dfe46e72d01/data_prep/spring_example_HPCs.ipynb

## Input variables

In [ ]:
sample_name = ['d2_1', 'd2_2','d2_3', 'd4_L_1','d4_L_2','d4_nBC', 'd4_R_1','d6_L_1','d6_L_2','d6_R_1','d6_R_2','d6_R_3']
min_tot = [1000 for s in sample_name] # initial guess for total transcript counts threshold
nSamp = len(sample_name)
input_path = "/home/mzo5929/Keerthana/LARRY/NewData/"


## Importing and installing pacakges

In [ ]:
import numpy as np
import scipy.sparse
import csv
import numpy as np
import networkx as nx
import pickle
import json
import pandas as pd
import gzip
import subprocess
import matplotlib.pyplot as plt


Function to load count matrix in .tsv format and parse through it

In [ ]:
def load_text(file_name_zipped, delim='\t', load_cell_bcs=True):
    command = ["gunzip", file_name_zipped]
    file_name = file_name_zipped[:-3]
    subprocess.run(command)
    with open(file_name) as csvfile:
        file_data = csv.reader(csvfile, delimiter=delim, quotechar='|')
        headers = next(file_data)
        X_row = []
        X_col = []
        X_data = []
        cell_bcs = []

        start_column = -1
        start_row = -1
        ncol = 0

        for row_ix, dat in enumerate(file_data):
            dat = [x.strip() for x in dat]  # Apply strip to each element in the list
            dat = [x for x in dat if x]  # Remove empty strings

            if start_row == -1:
                current_col = next((i for i, x in enumerate(dat) if is_float(x)), None)

                if current_col is not None:
                    start_row = row_ix
                    start_column = current_col

                    try:
                        rowdat = np.array(list(map(float, dat[start_column:])))
                    except ValueError:
                        print(f"Non-numeric value found in row {row_ix}: {dat[current_col:]}")
                        continue

                    ncol = len(rowdat)
                    non_zero_indices = np.nonzero(rowdat)[0]

                    X_col.extend(non_zero_indices)
                    X_row.extend([row_ix - start_row] * len(non_zero_indices))
                    X_data.extend(rowdat[non_zero_indices])

                    if load_cell_bcs:
                        cell_bcs.append(dat[0])
            else:
                try:
                    if load_cell_bcs:
                        cell_bcs.append(dat[0])

                    rowdat = np.array(list(map(float, dat[start_column:])))
                    if len(rowdat) != ncol:
                        raise ValueError('Rows have different numbers of numeric columns.')

                    non_zero_indices = np.nonzero(rowdat)[0]
                    X_col.extend(non_zero_indices)
                    X_row.extend([row_ix - start_row] * len(non_zero_indices))
                    X_data.extend(rowdat[non_zero_indices])

                except ValueError:
                    print(f"Non-numeric value found in row {row_ix}: {dat[start_column:]}")
                    continue

        if start_row == -1:
            return 'ERROR: no numeric values found'

        nrow = row_ix - start_row + 1
        E = scipy.sparse.coo_matrix((X_data, (X_row, X_col)), dtype=float, shape=(nrow, ncol)).tocsc()
        return E, np.array(cell_bcs), headers

def is_float(value):
    try:
        float(value)
        return True
    except ValueError:
        return False

Loading count matrices from .tsv.gz or .tsv whichever is faster. 

In [ ]:
D = {}
gene_list = {}
for j,s in enumerate(sample_name):
    D[s] = {}
    gene_list[s] = {}
    D[s]['meta'] = {'min_tot': min_tot[j]}

In [ ]:
for s in sample_name:
    print ('_________________', s)
    print('Loading from text file')
    result = load_text(input_path + s + "/" + s + '.counts.tsv.gz', delim='\t', load_cell_bcs=True)

    if isinstance(result, str):
        # Handle the error message
        print("Error:", result)
    else:
        # Unpack the result
        E, cell_bcs, gene_lists = result
        D[s]['E'] = E
        D[s]['cell_bcs'] = cell_bcs
        gene_list[s]['gene_list'] = gene_lists
        print(D[s]['E'].shape)
    

## Filter cells by total counts (based on numbers on table 1 of the paper)

In [145]:
sample_name

['d2_1',
 'd2_2',
 'd2_3',
 'd4_L_1',
 'd4_L_2',
 'd4_nBC',
 'd4_R_1',
 'd6_L_1',
 'd6_L_2',
 'd6_R_1',
 'd6_R_2',
 'd6_R_3']

In [ ]:
# adjust total counts thresholds - 
UMI_cutoffs = [1500, 1500, 1500, ]
for s in sample_name:
    D[s]['meta']['min_tot'] = 100
    D[s]['total_counts'] = np.sum(D[s]['E'], axis=1).A[:,0]

    fig = plt.figure()
    ax = fig.add_subplot(111)
    ax.hist(D[s]['total_counts'], bins=np.logspace(0, 6, 50))
    ax.set_xscale('log')
    ax.set_xlabel('Transcripts per barcode')
    ax.set_ylabel('Number of barcodes')
    ax.plot([D[s]['meta']['min_tot'],D[s]['meta']['min_tot']],ax.get_ylim());


    ix = D[s]['total_counts'] >= D[s]['meta']['min_tot']
    print(s, np.sum(ix), '/', D[s]['E'].shape[0], np.median(D[s]['total_counts'][ix]), np.mean(D[s]['total_counts'][ix]))

In [146]:
cell_bcs

['bcFYIE',
 'bcESVS',
 'bcADPN',
 'bcFRUU',
 'bcBWXD',
 'bcHEZC',
 'bcHSUF',
 'bcCZYM',
 'bcEKIE',
 'bcHUUM',
 'bcHOVC',
 'bcFSSG',
 'bcBNVE',
 'bcEKIP',
 'bcGUUI',
 'bcABCS',
 'bcFVRT',
 'bcAUMC',
 'bcAOKV',
 'bcGKCA',
 'bcANTU',
 'bcALML',
 'bcEWTT',
 'bcCZBN',
 'bcHHWO',
 'bcDOBW',
 'bcHHRU',
 'bcFHTK',
 'bcGUDZ',
 'bcCLQT',
 'bcGXQI',
 'bcCNYE',
 'bcEFTZ',
 'bcFUWN',
 'bcHPLN',
 'bcEBLE',
 'bcGBSN',
 'bcGGCE',
 'bcDCCW',
 'bcEWSF',
 'bcDDIQ',
 'bcECVX',
 'bcHTGA',
 'bcGIIT',
 'bcBDWX',
 'bcERAU',
 'bcCQTH',
 'bcFSRO',
 'bcBMYA',
 'bcAYZV',
 'bcFJTD',
 'bcIJFZ',
 'bcGELP',
 'bcBJSF',
 'bcBFUB',
 'bcDLKD',
 'bcBFGV',
 'bcDLCX',
 'bcENDB',
 'bcHKQJ',
 'bcGNWM',
 'bcBGAV',
 'bcCSAR',
 'bcARWE',
 'bcGPEB',
 'bcBEPI',
 'bcGFEQ',
 'bcFWWM',
 'bcFERH',
 'bcAASE',
 'bcFZRW',
 'bcGRFV',
 'bcADLL',
 'bcEQLY',
 'bcBTVH',
 'bcGAXP',
 'bcDRFD',
 'bcAQMP',
 'bcDKDS',
 'bcCHOX',
 'bcBSJC',
 'bcAFKE',
 'bcFIQG',
 'bcHKBL',
 'bcAFYZ',
 'bcEYNB',
 'bcAFRG',
 'bcEHXX',
 'bcBURZ',
 'bcGRDS',
 'bcFMJB',

Function to filter the cells based on counts

In [ ]:
def filter_dict(d, filt):
    for k,v in d.items():
        if k != 'meta':
            if len(v.shape) == 1:
                d[k] = v[filt]
            else:
                d[k] = v[filt,:]
    return d

In [ ]:
for s in sample_name:
    print('---  %s ---' %s)
    print('Pre-filter: %i barcodes' %D[s]['E'].shape[0])
    D[s]['cell_index'] = np.arange(D[s]['E'].shape[0])
    tmpfilt = np.nonzero(D[s]['total_counts'] >= D[s]['meta']['min_tot'])[0]
    D[s] = filter_dict(D[s], tmpfilt)
    print('Post-filter: %i barcodes' %D[s]['E'].shape[0])
    
del tmpfilt

## Filtering by mitochondrial fractions

Identifying the mito genes

In [ ]:
mt_ix = [i for i, g in enumerate(gene_lists) if g.startswith('mt-')] 
print([gene_lists[i] for i in mt_ix])
mt_ix = [i - 1 for i in mt_ix]

In [ ]:
# Assuming D[s]['E'] is a sparse matrix (e.g., scipy.sparse.coo_matrix)
# and mt_ix contains the indices of mitochondrial genes

# Ensure that mt_ix is within valid bounds
if max(mt_ix) >= D[s]['E'].shape[1]:
    raise ValueError("Invalid indices in mt_ix")

# Calculate the sum of mitochondrial genes along each row
mt_sums = D[s]['E'][:, mt_ix].sum(axis=1).A.flatten()

# Ensure that the sum is within valid bounds
if len(mt_sums) != D[s]['E'].shape[0]:
    raise ValueError("Invalid result size for mt_sums")

# Calculate the total sum of expression values along each row
total_sums = D[s]['E'].sum(axis=1, dtype=float).A.flatten()

# Ensure that the total sum is within valid bounds
if len(total_sums) != D[s]['E'].shape[0]:
    raise ValueError("Invalid result size for total_sums")

# Avoid division by zero by handling cases where total_sums is zero
D[s]['mito_frac'] = np.divide(mt_sums, total_sums, out=np.zeros_like(mt_sums), where=(total_sums != 0))


Plotting the mito gene fraction in each cell to use for filtering - using 20% from the methods of the paper

In [ ]:
# set mito-gene frac threshold
for s in sample_name:
    D[s]['meta']['max_mt'] = 0.2

for s in sample_name:
    fig = plt.figure(figsize=(8, 6))
    ax = fig.add_subplot(111, xscale='linear', yscale='linear',
        xlabel='MT frac.', ylabel='no. cells')

    D[s]['mito_frac'] = np.sum(D[s]['E'][:,mt_ix], axis=1).A[:,0] / np.sum(D[s]['E'], axis=1,dtype=float).A[:,0]

    ax.hist(D[s]['mito_frac'], cumulative=False, 
            bins=np.linspace(0, 1, 40))

    ax.plot([D[s]['meta']['max_mt'],D[s]['meta']['max_mt']],ax.get_ylim());
    print(D[s]['E'].shape[0], np.sum(D[s]['mito_frac'] <= D[s]['meta']['max_mt']))

Actually removing cells with high mitochondrial content

In [ ]:
for s in sample_name:
    print('---  %s ---' %s)
    print('Pre-filter: %i barcodes' %D[s]['E'].shape[0])
    tmpfilt = np.nonzero(D[s]['mito_frac'] <= D[s]['meta']['max_mt'])[0]
    D[s] = filter_dict(D[s], tmpfilt)
    print('Post-filter: %i barcodes' %D[s]['E'].shape[0])
    
del tmpfilt

## Merge and Normalise

In [ ]:
# create master dataset (all SPRING subsets will refer back to this)
# build sample index
samp_id_flat = np.array([],dtype=str)
for s in sample_name:
    samp_id_flat = np.append(samp_id_flat, [s] * D[s]['E'].shape[0])

# build flat cell BC list
cell_bcs_flat = np.array([],dtype=str)
for s in sample_name:
    cell_bcs_flat = np.append(cell_bcs_flat, D[s]['cell_bcs'])

# merge quality metrics
total_counts = np.zeros(len(samp_id_flat), dtype=int)
mito_frac = np.zeros(len(samp_id_flat), dtype=float)
for s in sample_name:
    total_counts[samp_id_flat == s] = D[s]['total_counts']
    mito_frac[samp_id_flat == s] = D[s]['mito_frac']

# merge counts matrices
E = scipy.sparse.vstack([D[s]['E'] for s in sample_name]).tocsc()

Normalising the data

In [ ]:
#Function to normalise
def tot_counts_norm(E, exclude_dominant_frac = 1, included = [], target_mean = 0):
    ''' 
    Cell-level total counts normalization of input counts matrix, excluding overly abundant genes if desired.
    Return normalized counts, average total counts, and (if exclude_dominant_frac < 1) list of genes used to calculate total counts 
    '''

    E = E.tocsc()
    ncell = E.shape[0]
    if len(included) == 0:
        if exclude_dominant_frac == 1:
            tots_use = E.sum(axis=1)
        else:
            tots = E.sum(axis=1)
            wtmp = scipy.sparse.lil_matrix((ncell, ncell))
            wtmp.setdiag(1. / tots)
            included = np.asarray(~(((wtmp * E) > exclude_dominant_frac).sum(axis=0) > 0))[0,:]
            tots_use = E[:,included].sum(axis = 1)
            #print 'Excluded %i genes from normalization' %(np.sum(~included))
    else:
        tots_use = E[:,included].sum(axis = 1)

    if target_mean == 0:
        target_mean = np.mean(tots_use)

    w = scipy.sparse.lil_matrix((ncell, ncell))
    w.setdiag(float(target_mean) / tots_use)
    Enorm = w * E

    return Enorm.tocsc(), target_mean, included

In [ ]:
# normalize by total counts
E = tot_counts_norm(E)[0]

## Saving the files needed for next step

In [ ]:
np.savetxt('genes.txt', gene_lists, fmt='%s')
np.savetxt('samp_id_flat.txt', samp_id_flat, fmt='%s')
np.savetxt('cell_bcs_flat.txt', cell_bcs_flat, fmt='%s')
np.savetxt('total_counts.txt', total_counts)

## Converting the .tsv counts matrix to .mtx with barcodes.tsv.gz and genes.tsv.gz

## Things to do:
    1. Split the matrix, cell IDs by sample
    2. Create separate directories for each sample
    3. Save the .mtx and the 2 .tsv.gz in it

### Function to convert a list to tsv.gz

In [ ]:
def list_to_tsv_gz(list1, filename):
    # Converts a list to a .tsv.gz file.
    with gzip.open(filename, "wt", newline="", encoding="utf-8") as f:
        writer = csv.writer(f, delimiter="\t")
        for item in list1:
            writer.writerow([item]) 

### Creating the gene list - common list for all samples

In [136]:
# %cd ..
my_file = open("genes.txt", "r") 
data = my_file.read() 
gene_list_convert = data.split("\n") 
my_file.close()

## Iterating over samples to save the files

In [143]:
# %cd 10X
import os
for s in sample_name:
    if  not(os.path.exists(s)):
        subprocess.run(["mkdir", s])
    mtx_file_name =  s + "/matrix.mtx"
    scipy.io.mmwrite(mtx_file_name, D[s]['E'])
    list_to_tsv_gz(D[s]['cell_bcs'], s + "/barcodes.tsv.gz")
    list_to_tsv_gz(gene_list_convert[1:], s + "/genes.tsv.gz")
    

# This is for clonal - mapping fully adapted from: https://github.com/AllonKleinLab/LARRY/blob/master/clonal_annotation.ipynb

## Setting parameters and input paths

In [ ]:
N_READS = 10
N_UMIS = 1
N_HAMMING = 3
CELL_BCS_PATH = 'cell_bcs_flat.txt'
LIB_NAMES_PATH = 'samp_id_flat.txt'

## Opening required files

In [ ]:
# From the above steps
cell_bcs = open(CELL_BCS_PATH).read().strip('\n').split('\n')
lib_names = open(LIB_NAMES_PATH).read().strip('\n').split('\n')

#From step 1 of LARRY
counts = {}
f = gzip.open('LARRY_sorted_and_filtered_barcodes.fastq.gz')
l = f.readline().decode("utf-8").strip('\n')
current_tag = []
i = 0
print('Reading in all barcodes')
while not (l == '' and len(current_tag)==0):
    i += 1
    if i % (3*10**6)==0: print('Processed '+repr(int(i/3))+' reads')
    if l == '':
        current_tag = []
    elif l[0] == '>':
        current_tag = l[1:].split(',')
    elif l != '' and len(current_tag)==3:
        current_tag.append(l)
        current_tag = tuple(current_tag)
        if not current_tag in counts: counts[current_tag] = 0
        counts[current_tag] += 1
    l = f.readline().decode("utf-8").strip('\n')

In [ ]:
len(counts)

In [ ]:
num_reads = [v for k,v in counts.items()]
plt.hist(np.log(num_reads)/np.log(10), bins=50)
plt.plot([np.log(N_READS)/np.log(10),np.log(N_READS)/np.log(N_READS)],[1,10**5],'-k',linewidth=2)
plt.text(np.log(N_READS)/np.log(10)*1.1,10**5*.8,'N_READS cutoff', fontsize=12)
plt.yscale('log')

counts_filtered = {k:v for k,v in counts.items() if v >= N_READS}
print('Retaining '+repr(len(counts_filtered))+ ' out of '+repr(len(counts))+' (Sample,Cell-BC,UMI,GFP-BC) combinations')

## Collapse GFP-BCs by hamming distance


In [ ]:
def hamming(bc1,bc2): return np.sum([x1 != x2 for x1,x2 in zip(bc1,bc2)])

all_gfp_bcs = sorted(set([k[3] for k in counts_filtered]))
good_gfp_bcs = []
bc_map = {}
for i,bc1 in enumerate(all_gfp_bcs):
    if i > 0 and i % 500 == 0: print('Mapped '+repr(i)+' out of '+repr(len(all_gfp_bcs))+' barcodes')
    mapped = False
    for bc2 in good_gfp_bcs:
        if hamming(bc1,bc2) <= N_HAMMING:
            mapped = True
            bc_map[bc1] = bc2
            break
    if not mapped:
        good_gfp_bcs.append(bc1)

print('\nCollapsed '+repr(len(bc_map))+' barcodes')
for bc in good_gfp_bcs: bc_map[bc] = bc

## Counting UMIs in (Cells, Barcodes)

In [ ]:
cell_data = {}
for lib,cell in zip(lib_names,cell_bcs):
    cell_data[(lib,cell)] = {}

for lib,cell,umi,BC in counts_filtered.keys():
    if (lib,cell) in cell_data:
        if not BC in cell_data[(lib,cell)]:
            cell_data[(lib,cell)][BC] = 0
        cell_data[(lib,cell)][BC] += 1
cell_data
# BC_lists = []
# for i in range(1,10):
#     BC_list = []
#     for lib,cell in zip(lib_names,cell_bcs):
#         bc_counts = cell_data[(lib,cell)]
#         valid_bcs = [bc_map[k] for k,v in bc_counts.items() if v >= i]
#         BC_list.append(''.join(sorted(valid_bcs)))
#     BC_lists.append(BC_list)

# efficiency = np.array([len([ll for ll in l if len(ll)>0]) for l in BC_lists]) / len(cell_bcs)
# plt.plot(range(1,10),efficiency)
# plt.plot([N_UMIS,N_UMIS],[np.min(efficiency),np.max(efficiency)],'-k',linewidth=2)
# plt.text(N_UMIS*1.1,np.max(efficiency)*.95,'UMI cutoff',fontsize=14)
# final_BCs = BC_lists[N_UMIS-1]

# print('\nFinal annotation has '+repr(len(set(final_BCs)))+' clones in '+repr(len([l for l in final_BCs if len(l)>0]))+' cells')


In [ ]:
data_list = [{'Library': key[0], 'Cell': key[1], 'Barcodes': value} for key, value in cell_data.items()]
df = pd.DataFrame(data_list)

rows = []
for _, row in df.iterrows():
    if not row['Barcodes']: 
        continue
    for barcode, count in row['Barcodes'].items():
        rows.append([row['Library'], row['Cell'], barcode, count])

result_df = pd.DataFrame(rows, columns=['sample', 'cellID', 'barcode', 'Counts'])
print(np.sum(result_df['Counts']))

In [ ]:
df_expanded = result_df.loc[np.repeat(result_df.index, result_df['Counts'])].reset_index(drop=True)

# Drop the 'Counts' column
df_expanded = df_expanded.drop(columns='Counts')
`
df_expanded.to_csv("LARRY_Set1_Barcodes.csv",index=False)

In [ ]:
import pandas as pd
# %cd NewData

obj = pd.read_pickle('d2_1/abundant_barcodes.pickle')
obj_2 = pd.read_pickle('d2_2/abundant_barcodes.pickle')
intersection_keys = obj.keys() & obj_2.keys()
difference_keys = set(obj.keys()) - set(intersection_keys)
print(difference_keys)

In [168]:
df = pd.read_csv("/home/mzo5929/Keerthana/LARRY/oldRawData/stateFate_inVitro_metadata.txt", sep="\t")

In [175]:
df["Library"].unique()


array(['d6_2_2', 'd6_2_3', 'd6_2_1', 'd6_1_1', 'd4_2_1', 'd2_3', 'd2_2',
       'd2_1', 'd4_2_2', 'd4_2_4', 'd4_1_2', 'd4_1_1', 'd6_1_2',
       'LK_d6_2_2', 'LK_d6_2_1', 'LSK_d2_2', 'LSK_d2_3', 'LSK_d2_1',
       'LK_d4_1', 'LK_d4_2', 'LK_d2', 'LSK_d6_2_3', 'LSK_d6_2_2',
       'LSK_d6_2_1', 'LSK_d4_1_2', 'LSK_d4_1_3', 'LSK_d4_1_1',
       'LSK_d4_2_1', 'LSK_d4_2_3', 'LSK_d4_2_2', 'LK_d6_1_1', 'LK_d6_1_2',
       'LSK_d6_1_1', 'LSK_d6_1_2', 'LSK_d6_1_3'], dtype=object)